### 库导入

In [1]:
import os
import numpy as np
import pandas as pd
import mne
from scipy import signal
import random
import matplotlib.pyplot as plt
%matplotlib qt

### 随机抽取一个正样本一个负样本

In [2]:
def get_random_samples(positive_file, negative_file):
    with open(positive_file, 'r') as f:
        positive_samples = f.readlines()
    with open(negative_file, 'r') as f:
        negative_samples = f.readlines()
    
    positive_index = random.randint(0, len(positive_samples)//4) * 4
    negative_index = random.randint(0, len(negative_samples)//4) * 4
    
    positive_sample = positive_samples[positive_index:positive_index+4]
    negative_sample = negative_samples[negative_index:negative_index+4]
    
    # 提取文件名和时间信息
    positive_file_name = positive_sample[0].split(': ')[1].strip()
    positive_start_time = int(positive_sample[1].split(': ')[1].strip())
    positive_end_time = int(positive_sample[2].split(': ')[1].strip())

    negative_file_name = negative_sample[0].split(': ')[1].strip()
    negative_start_time = int(negative_sample[1].split(': ')[1].strip())
    negative_end_time = int(negative_sample[2].split(': ')[1].strip())
    
    return positive_file_name, positive_start_time, positive_end_time, negative_file_name, negative_start_time, negative_end_time



### 读取和可视化

In [3]:
def load_and_visualize_samples(positive_file, negative_file, data_positive_folder, data_negative_folder):
    # 获取正样本和负样本的文件名和时间点
    positive_file_name, positive_start_time, positive_end_time, negative_file_name, negative_start_time, negative_end_time = get_random_samples(positive_file, negative_file)

    # 显示正样本和负样本的文件名和时间点
    print("Positive Sample:")
    print("File Name:", positive_file_name)
    print("Start Time:", positive_start_time)
    print("End Time:", positive_end_time)
    print()
    print("Negative Sample:")
    print("File Name:", negative_file_name)
    print("Start Time:", negative_start_time)
    print("End Time:", negative_end_time)
    print()

    # 读取正样本和负样本的数据
    positive_data_file = os.path.join(data_positive_folder, positive_file_name)
    negative_data_file = os.path.join(data_negative_folder, negative_file_name)

    positive_raw = mne.io.read_raw_edf(positive_data_file)
    negative_raw = mne.io.read_raw_edf(negative_data_file)

    # 使用raw.plot函数可视化数据
    positive_raw.plot()
    negative_raw.plot()


### 调用读取

In [4]:
data_positive_folder = 'D:/Epilepsy/data_positive_original'
data_negative_folder = 'D:/Epilepsy/data_negative_original'
positive_file = 'combined_positive.txt'
negative_file = 'combined_negative.txt'

load_and_visualize_samples(positive_file, negative_file, data_positive_folder, data_negative_folder)


Positive Sample:
File Name: chb19_28.edf
Start Time: 335
End Time: 341

Negative Sample:
File Name: chb14_20.edf
Start Time: 696
End Time: 702

Extracting EDF parameters from D:\Epilepsy\data_positive_original\chb19_28.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\Epilepsy\data_negative_original\chb14_20.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Using matplotlib as 2D backend.


C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\2708482990.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  positive_raw = mne.io.read_raw_edf(positive_data_file)
C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\2708482990.py:21: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  positive_raw = mne.io.read_raw_edf(positive_data_file)
C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\2708482990.py:22: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  negative_raw = mne.io.read_raw_edf(negative_data_file)
C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\2708482990.py:22: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  negative_raw = mne.io.read_raw_edf(negative_data_file)


### 低通滤波

In [7]:
import os
import numpy as np
import scipy.signal as signal
import mne

def load_and_filter_samples(positive_file, negative_file, data_positive_folder, data_negative_folder, filtered_positive_folder, filtered_negative_folder):
    # 获取正样本和负样本的文件名和时间点
    positive_file_name, positive_start_time, positive_end_time, negative_file_name, negative_start_time, negative_end_time = get_random_samples(positive_file, negative_file)

    # 显示正样本和负样本的文件名和时间点
    print("Positive Sample:")
    print("File Name:", positive_file_name)
    print("Start Time:", positive_start_time)
    print("End Time:", positive_end_time)
    print()
    print("Negative Sample:")
    print("File Name:", negative_file_name)
    print("Start Time:", negative_start_time)
    print("End Time:", negative_end_time)
    print()

    # 读取正样本和负样本的数据
    positive_data_file = os.path.join(data_positive_folder, positive_file_name)
    negative_data_file = os.path.join(data_negative_folder, negative_file_name)

    positive_raw = mne.io.read_raw_edf(positive_data_file, preload=True)
    negative_raw = mne.io.read_raw_edf(negative_data_file, preload=True)

    # 截取
    positive_crop = positive_raw.copy().crop(positive_start_time, positive_end_time)
    negative_crop = negative_raw.copy().crop(negative_start_time, negative_end_time)
    
    # 滤波
    fs = positive_crop.info['sfreq']
    lowcut = 0.01
    highcut = 32

    # 设计四阶Butterworth滤波器
    b, a = signal.butter(4, [lowcut, highcut], btype='band', fs=fs)
    
    # 应用滤波器
    positive_filtered_data = signal.filtfilt(b, a, positive_crop.get_data())
    negative_filtered_data = signal.filtfilt(b, a, negative_crop.get_data())
    
   # 保存滤波后的样本
    positive_file_base = os.path.basename(positive_data_file).replace('.edf', '')
    negative_file_base = os.path.basename(negative_data_file).replace('.edf', '')
    positive_save_name = f"{positive_file_base}_{positive_start_time}_filtered.npy"
    negative_save_name = f"{negative_file_base}_{negative_start_time}_filtered.npy"
    
    filtered_positive_file = os.path.join(filtered_positive_folder, positive_save_name)
    filtered_negative_file = os.path.join(filtered_negative_folder, negative_save_name)

    np.save(filtered_positive_file, positive_filtered_data)
    np.save(filtered_negative_file, negative_filtered_data)


### 循环调用N次读取并滤波 得到N个正负样本


In [6]:
data_positive_folder = 'D:/Epilepsy/data_positive_original'
data_negative_folder = 'D:/Epilepsy/data_negative_original'
positive_file = 'combined_positive.txt'
negative_file = 'combined_negative.txt'
filtered_positive_folder = r'D:\Epilepsy\filtered_positive'
filtered_negative_folder = r'D:\Epilepsy\filtered_negative'
num_iterations = 100  # 指定循环次数N

for _ in range(num_iterations):
    load_and_filter_samples(positive_file, negative_file, data_positive_folder, data_negative_folder, filtered_positive_folder, filtered_negative_folder)


Positive Sample:
File Name: chb05_16.edf
Start Time: 2401
End Time: 2407

Negative Sample:
File Name: chb22_17.edf
Start Time: 2856
End Time: 2862

Extracting EDF parameters from D:\Epilepsy\data_positive_original\chb05_16.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\220387253.py:26: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  positive_raw = mne.io.read_raw_edf(positive_data_file, preload=True)


Extracting EDF parameters from D:\Epilepsy\data_negative_original\chb22_17.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\220387253.py:27: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  negative_raw = mne.io.read_raw_edf(negative_data_file, preload=True)


Positive Sample:
File Name: chb14_04.edf
Start Time: 2823
End Time: 2829

Negative Sample:
File Name: chb20_21.edf
Start Time: 0
End Time: 6

Extracting EDF parameters from D:\Epilepsy\data_positive_original\chb14_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

Extracting EDF parameters from D:\Epilepsy\data_negative_original\chb20_21.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 926207  =      0.000 ...  3617.996 secs...


C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\220387253.py:27: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '.'}. Applying running numbers for duplicates.
  negative_raw = mne.io.read_raw_edf(negative_data_file, preload=True)


Positive Sample:
File Name: chb07_19.edf
Start Time: 13700
End Time: 13706

Negative Sample:
File Name: chb02_23.edf
Start Time: 1218
End Time: 1224

Extracting EDF parameters from D:\Epilepsy\data_positive_original\chb07_19.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3689215  =      0.000 ... 14410.996 secs...


C:\Users\Litchi\AppData\Local\Temp\ipykernel_8492\220387253.py:26: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  positive_raw = mne.io.read_raw_edf(positive_data_file, preload=True)


KeyboardInterrupt: 

### 特征提取函数

In [13]:
from __future__ import division
import numpy as np
np.set_printoptions(threshold=np.inf)
import pywt
import math
import nolds
from pyentrp import entropy as ent
from scipy.stats import skew
from scipy.stats import kurtosis

#time domain  
def min_X(X):
    return np.min(X)

def max_X(X):
    return np.max(X)

def std_X(X):
    return np.std(X)

def mean_X(X):
    return np.mean(X)

def var_X(X):
    return np.var(X)

def totalVariation(X):
    Max = np.max(X)
    Min = np.min(X)
    return np.sum(np.abs(np.diff(X)))/((Max-Min)*(len(X)-1))

#偏度用来度量分布是否对称。正态分布左右是对称的，偏度系数为0。较大的正值表明该分布具有右侧较长尾部。较大的负值表明有左侧较长尾部
def skew_X(X):
    skewness = skew(X)
    return skewness

#峰度系数（Kurtosis）用来度量数据在中心聚集程度。在正态分布情况下，峰度系数值是3。
#>3的峰度系数说明观察量更集中，有比正态分布更短的尾部；
#<3的峰度系数说明观测量不那么集中，有比正态分布更长的尾部，类似于矩形的均匀分布。
def kurs_X(X):
    kurs = kurtosis(X)
    return kurs

#常用均方根值来分析噪声
def rms_X(X):
    RMS = np.sqrt((np.sum(np.square(X))) * 1.0 / len(X))
    return RMS

def peak_X(X):
    Peak = np.max([np.abs(max_X(X)), np.abs(min_X(X))])
    return Peak

#峰均比
def papr_X(X):
    Peak = peak_X(X)
    RMS = rms_X(X)
    PAPR = np.square(Peak) * 1.0 / np.square(RMS)
    return PAPR

'''
计算时域10个bin特征
'''
def filter_X(X):
    X_new = []
    length = np.shape(X)[0]
    for i in range(1, length):
        if i != 0 and np.array_equal(X[i], X[i-1]):
            continue
        X_new.append(X[i])
    return X_new


#求X中所有的极大值和极小值点
def minmax_cal(X):
    length = np.shape(X)[0]
    min_value = []
    min_index = []
    max_value = []
    max_index = []
    first = ''
    for i in range(1, length-1):
        if np.any(X[i] < X[i-1]) and np.any(X[i] < X[i+1]):
            min_value.append(X[i])
            min_index.append(i)
        if np.any(X[i] > X[i-1]) and np.any(X[i] > X[i+1]):
            max_value.append(X[i])
            max_index.append(i)
    if len(min_index) and len(max_index):       
        if max_index[0] > min_index[0]:
            first = 'min'
        else:
            first = 'max'
        return min_value, max_value, first
    else:
        return [],[],[]


#计算所有的极大值和极小值的差值        
def minmax_sub_cal(X):
    min_value, max_value, first = minmax_cal(X)
    if min_value and max_value and first:
        max_length = np.shape(max_value)[0]
        sub = []
        if first == 'min':
            for i in range(max_length-1):
                sub.append(max_value[i] - min_value[i])
                sub.append(max_value[i] - min_value[i+1])
        else:
            for i in range(1, max_length-1):
                sub.append(max_value[i] - min_value[i-1])
                sub.append(max_value[i] - min_value[i])   
        return sub
    else:
        return None     

#计算极大极小值差值占比            
def minmax_percent_cal(X, step=10):  
    X = filter_X(X)
    sub = minmax_sub_cal(X)
    if sub:
        length = int(np.shape(sub)[0])
        max_value = np.max(sub)
        min_value = np.min(sub)
        diff = max_value - min_value
        value = diff / step
        nums = []
        sub = np.array(sub)
        for i in range(step):
            scale_min = sub >= min_value + i * value
            scale_max = sub < min_value + (i + 1) * value
            scale = scale_min & scale_max
            num = np.where(scale)[0]
            size = np.shape(num)[0]
            nums.append(size)
        nums[-1] = nums[-1] + np.sum(sub == max_value)
        per = np.array(nums, dtype=int) / length
        return per
    else:
        return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#非线性特征提取
#采样频率为256Hz,则信号的最大频率为128Hz，进行5层小波分解
def relativePower(X):
    Ca5, Cd5, Cd4, Cd3, Cd2, Cd1 = pywt.wavedec(X, wavelet='db4', level=5)
    min_length = min(len(Ca5), len(Cd5), len(Cd4), len(Cd3), len(Cd2), len(Cd1))
    Ca5 = Ca5[:min_length]
    Cd5 = Cd5[:min_length]
    Cd4 = Cd4[:min_length]
    Cd3 = Cd3[:min_length]
    Cd2 = Cd2[:min_length]
    Cd1 = Cd1[:min_length]
    EA5 = sum([i*i for i in Ca5])
    ED5 = sum([i*i for i in Cd5])
    ED4 = sum([i*i for i in Cd4])
    ED3 = sum([i*i for i in Cd3])
    ED2 = sum([i*i for i in Cd2])
    ED1 = sum([i*i for i in Cd1])
    E = EA5 + ED5 + ED4 + ED3 + ED2 + ED1
    pEA5 = EA5/E
    pED5 = ED5/E
    pED4 = ED4/E
    pED3 = ED3/E
    pED2 = ED2/E
    pED1 = ED1/E
    return pEA5, pED5, pED4, pED3, pED2, pED1

#nonlinear analysis
#小波熵
def wavelet_entopy(X):
    [pEA5, pED5, pED4, pED3, pED2, pED1] = relativePower(X)
    wavelet_entopy = - (pEA5*math.log(pEA5) + pED5*math.log(pED5)
    + pED4*math.log(pED4) + pED3*math.log(pED3) + pED2*math.log(pED2) + pED1*math.log(pED1))
    return wavelet_entopy

#计算Detrended Fluctuation Analysis值
def DFA(X):
    y = nolds.dfa(X)
    return y

#计算赫斯特指数
def Hurst(X):
    y = nolds.hurst_rs(X)
    return y

#计算Petrosian's Fractal Dimension分形维数值
def Petrosian_FD(X):
    D = np.diff(X)

    delta = 0;
    N = len(X)
    #number of sign changes in signal
    for i in range(1, len(D)):
        if (D[i] * D[i-1] < 0).any() < 0:
            delta += 1

    feature = np.log10(N) / (np.log10(N) + np.log10(N / (N + 0.4 * delta)))

    return feature

#计算样本熵
def sample_entropy(X):
    y = nolds.sampen(X)
    return y

#计算排列熵
#度量时间序列复杂性的一种方法,排列熵H的大小表征时间序列的随机程度，值越小说明该时间序列越规则，反之，该时间序列越具有随机性。
def permutation_entropy(X):
    y = ent.permutation_entropy(X, 4, 1)
    return y

#Hjorth Parameter: mobility and complexity
def Hjorth(X):
    D = np.diff(X)
    D = list(D)
    D.insert(0, X[0])
    VarX = np.var(X)
    VarD = np.var(D)
    Mobility = np.sqrt(VarD / VarX)

    DD = np.diff(D)
    VarDD = np.var(DD)
    Complexity = np.sqrt(VarDD / VarD) / Mobility

    return Mobility, Complexity

In [18]:
import os
import numpy as np
import pandas as pd

def extract_features(X):
    features = []
    features.append(min_X(X))
    features.append(max_X(X))
    features.append(std_X(X))
    features.append(mean_X(X))
    features.append(var_X(X))
    features.append(totalVariation(X))
    features.append(skew_X(X))
    features.append(kurs_X(X))
    features.append(rms_X(X))
    min_value, max_value, first = minmax_cal(X)

    if min_value is None or max_value is None:
        return None
    
    if len(min_value) != len(max_value):
        return None
    
    features.extend(min_value)
    features.extend(max_value)
    features.append(first)
    features.append(peak_X(X))
    features.append(papr_X(X))

    # Call minmax_percent_cal function and add its individual elements to features
    sub = minmax_percent_cal(X)
    features.extend(sub)

    # Fill missing features with zeros
    while len(features) < 17:
        features.append(0)

    # Truncate excess features
    features = features[:17]

    features.append(DFA(X))
    features.append(Petrosian_FD(X))
    features.append(sample_entropy(X))


    return features


# 滤波后样本文件夹路径
filtered_positive_folder = r'D:\Epilepsy\filtered_positive'
filtered_negative_folder = r'D:\Epilepsy\filtered_negative'

# 定义存储特征向量和标签的列表
X = []
y = []

# 遍历滤波后样本文件夹（正样本）
for file in os.listdir(filtered_positive_folder):
    if file.endswith('.npy'):
        # 加载滤波后的样本数据
        file_path = os.path.join(filtered_positive_folder, file)
        data = np.load(file_path)
        
        # 应用特征提取函数获取特征
        features = extract_features(data)
        
        # 检查特征向量是否为特殊值
        if features is None:
            # 特征向量不和谐，跳过这条数据
            continue
        
        # 添加特征向量和标签到列表中
        X.append(features)
        y.append(1)  # 正样本的标签为 1

# 遍历滤波后样本文件夹（负样本）
for file in os.listdir(filtered_negative_folder):
    if file.endswith('.npy'):
        # 加载滤波后的样本数据
        file_path = os.path.join(filtered_negative_folder, file)
        data = np.load(file_path)
        
        # 应用特征提取函数获取特征
        features = extract_features(data)
        
        # 检查特征向量是否为特殊值
        if features is None:
            # 特征向量不和谐，跳过这条数据
            continue
        
        # 添加特征向量和标签到列表中
        X.append(features)
        y.append(0)  # 负样本的标签为 0

# 将特征向量列表转换为DataFrame
X = pd.DataFrame(X)

# 转换标签列表为NumPy数组
y = np.array(y)

# 打印特征向量和标签
print("Features:")
print(X)
print("Labels:")
print(y)

# 检查每一列的特征值类型
for column in range(len(features)):
    column_values = [row[column] for row in data]
    column_type = type(column_values[0])
    print(f"Column {column}: Type = {column_type}")

# 将特征向量 X 和标签 y 保存为NumPy数组
X.to_numpy().dump('features.npy')
np.save('labels.npy', y)


Features:
           0         1         2         3             4          5   \
0   -0.000801  0.000832  0.000174 -0.000050  3.042424e-08   8.578171   
1   -0.000649  0.000790  0.000160 -0.000024  2.561761e-08  10.775748   
2   -0.000540  0.000710  0.000150 -0.000040  2.262825e-08   7.840534   
3   -0.000639  0.000644  0.000149 -0.000067  2.225268e-08   8.700020   
4   -0.000675  0.000575  0.000138  0.000028  1.915263e-08  14.234852   
..        ...       ...       ...       ...           ...        ...   
403 -0.000474  0.000695  0.000118 -0.000023  1.393997e-08   7.804528   
404 -0.000172  0.000117  0.000039 -0.000009  1.494249e-09  13.147220   
405 -0.000225  0.000121  0.000040 -0.000013  1.566719e-09   8.962627   
406 -0.000354  0.000268  0.000053 -0.000012  2.847753e-09   5.717284   
407 -0.000145  0.000270  0.000030  0.000003  9.245888e-10  10.008755   

                                                    6   \
0    [-0.27699353382943975, 0.12261703240544874, 0....   
1    [0.2

### 归一化特征矩阵

In [1]:
import numpy as np

# 加载特征矩阵
features = np.load('features.npy', allow_pickle=True)

# 检查每一列的特征值类型和形状
for i in range(features.shape[1]):
    column = features[:, i]
    column_type = type(column[0])
    column_shape = np.shape(column[0])
    print(f"Column {i}: Type = {column_type}, Shape = {column_shape}")
    
# 检查每一行的数组维度
for i in range(features.shape[0]):
    row = features[i, :]
    print(f"Row {i}: Shape = {np.shape(row)}")

# 最大-最小归一化
normalized_features = np.copy(features)  # 创建一个归一化后的特征矩阵副本

for i in range(features.shape[1]):
    column = features[:, i]
    
    if isinstance(column[0], np.ndarray):
        # 对多维度特征列进行归一化
        column_min = np.min(np.concatenate(column))
        column_max = np.max(np.concatenate(column))
        normalized_column = [(x - column_min) / (column_max - column_min) for x in column]
        normalized_features[:, i] = normalized_column
    else:
        # 单维度特征列不需要归一化
        pass

# 打印归一化后的特征矩阵
print(normalized_features)

# 保存归一化后的特征矩阵
np.save('normalized_features.npy', normalized_features)


Column 0: Type = <class 'float'>, Shape = ()
Column 1: Type = <class 'float'>, Shape = ()
Column 2: Type = <class 'float'>, Shape = ()
Column 3: Type = <class 'float'>, Shape = ()
Column 4: Type = <class 'float'>, Shape = ()
Column 5: Type = <class 'float'>, Shape = ()
Column 6: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 7: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 8: Type = <class 'float'>, Shape = ()
Column 9: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 10: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 11: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 12: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 13: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 14: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 15: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 16: Type = <class 'numpy.ndarray'>, Shape = (1537,)
Column 17: Type = <class 'float'>, Shape = ()
Column 18: Type = <class 'float'>, Shape = ()
Column


### 划分训练集和测试集


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 加载特征矩阵和标签数组
features = np.load('normalized_features.npy', allow_pickle=True)
labels = np.load('labels.npy', allow_pickle=True)

# 将多维数组展平为一维
flattened_features = []
for row in features:
    flattened_row = []
    for element in row:
        if isinstance(element, np.ndarray):
            flattened_row.extend(element.tolist())
        else:
            flattened_row.append(element)
    flattened_features.append(flattened_row)

# 转换为NumPy数组
flattened_features = np.array(flattened_features)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(flattened_features, labels, test_size=0.2, random_state=42)


# 打印训练集和测试集的形状
print("训练集特征矩阵形状：", X_train.shape)
print("训练集标签数组形状：", y_train.shape)
print("测试集特征矩阵形状：", X_test.shape)
print("测试集标签数组形状：", y_test.shape)


# 创建逻辑回归模型对象
model = LogisticRegression()

# 使用训练集进行模型训练
model.fit(X_train, y_train)

# 在测试集上进行预测
predictions = model.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, predictions)
print("准确率：", accuracy)

# 计算精确度
precision = precision_score(y_test, predictions)
print("精确度：", precision)

# 计算召回率
recall = recall_score(y_test, predictions)
print("召回率：", recall)

# 计算F1值
f1 = f1_score(y_test, predictions)
print("F1值：", f1)



训练集特征矩阵形状： (326, 15380)
训练集标签数组形状： (326,)
测试集特征矩阵形状： (82, 15380)
测试集标签数组形状： (82,)
准确率： 0.5853658536585366
精确度： 0.6571428571428571
召回率： 0.5111111111111111
F1值： 0.575


D:\Python\python3.11.3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
